In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Substitua pelo caminho correto se necessário
csv_path = '/content/drive/MyDrive/transacoes.csv'
df = pd.read_csv(csv_path)
df.head()

In [ ]:
# Pré-processamento simples
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['has_cbk'] = df['has_cbk'].fillna(False).astype(bool)

# Regras simples
def regras_simples(df):
    df['flag_muitas_transacoes'] = df.groupby('user_id')['timestamp'].transform(lambda x: x.diff().dt.total_seconds().fillna(9999) < 10)
    df['flag_valor_alto'] = df['transaction_amount'] > 10000
    df['flag_estorno'] = df['has_cbk']
    df['fraude_regra'] = df[['flag_muitas_transacoes', 'flag_valor_alto', 'flag_estorno']].any(axis=1)
    return df

df = regras_simples(df)
df['fraude_regra'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Features e label
features = ['transaction_amount', 'flag_muitas_transacoes', 'flag_valor_alto', 'flag_estorno']
df[features] = df[features].astype(float)
X = df[features]
y = df['fraude_regra'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escolha do modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
# modelo = XGBClassifier()
# modelo = LGBMClassifier()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print(classification_report(y_test, y_pred))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

In [ ]:
import joblib

# Exporta o modelo treinado
joblib.dump(modelo, '/content/drive/MyDrive/modelo_antifraude.pkl')
print("Modelo exportado com sucesso!")

In [ ]:
# Para rodar localmente: salve como app.py e execute com `flask run`
from flask import Flask, request, jsonify
import joblib
import pandas as pd

app = Flask(__name__)
modelo = joblib.load("modelo_antifraude.pkl")

@app.route('/predict', methods=['POST'])
def predict():
    dados = request.json
    df = pd.DataFrame([dados])
    pred = modelo.predict(df)[0]
    return jsonify({'fraude': bool(pred)})

# Para rodar localmente, descomente a linha abaixo
# app.run(host='0.0.0.0', port=5000)
